In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os
from google.cloud import bigquery

from datetime import timedelta
import warnings


# pd.set_option('display.max_columns', None)

os.chdir('/home/jupyter')
!pwd
# # %cd 
# !pwd
# (base) jupyter@starfire-1:~$ pip install jupyter-console
# (base) jupyter@starfire-1:~$ !jupyter console --existing
%connect_info



/home/jupyter
{
  "shell_port": 57565,
  "iopub_port": 49753,
  "stdin_port": 36911,
  "control_port": 56191,
  "hb_port": 58961,
  "ip": "127.0.0.1",
  "key": "036a6ac4-be89ebd356eeb7767b660dd0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "conda-root-py",
  "jupyter_session": "/home/jupyter/starfire_legacy/refinitiv_mna_deals-2.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-5270d709-2c4d-48e2-a39a-e36ecfab493e.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [5]:
# RDP_LOGIN = '<RDP Machine ID>'
# RDP_PASSWORD = '<RDP Password>'
# APP_KEY = '<APP Key>'
from IPython.display import HTML

import refinitiv.dataplatform as rdp

%run ./starfire_legacy/credentials.ipynb

session = rdp.open_platform_session(
    APP_KEY,
    rdp.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
)

# https://developers.lseg.com/en/article-catalog/article/getting-start-with-refinitiv-data-platform

In [29]:
# Download Deals data

master_df = pd.DataFrame()


from datetime import datetime, timedelta

# Initial start date
start_date = datetime.strptime('20180101', '%Y%m%d')

# Define the interval (15 days)
interval = timedelta(days=5)

# Initialize an empty list to store the date ranges
date_ranges = []

# Loop to create date intervals
for _ in range(10000):  # Adjust the range to get the desired number of intervals
    end_date = start_date + interval
    date_ranges.append((start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d')))
    start_date = end_date + timedelta(days=1)

# Print the date ranges
for start, end in date_ranges:
    success = False
    while not success:
        try:
            print(f"Start Date: {start}, End Date: {end}")

            print("Downloading data from refinitiv...")

            mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                        BETWEEN(TR.MnAAnnDate,{start},{end})/*dt:Date*/, \
                        CURN=USD)'


            # input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
            mnas = rdp.get_data(mna_screen,['TR.MNASDCDealNumber',
                                            'TR.MnAAnnDate', 'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                                            'TR.MnATarget', 'TR.MnATargetPermId',
                                            'TR.MnATargetPriTickerSym',
                                            'TR.MnATargetNation',
                                            'TR.MnATargetMacroInd',
                                            'TR.MnATargetMidInd',
                                            'TR.MnAAcquiror',
                                            'TR.MnAAcquirorPermId',
                                            'TR.MnAAcquirorMacroInd',
                                            'TR.MnAAcquirorMidInd',
                                            'TR.MnAAcquirorNation',
                                            'TR.MNADealId','TR.MnASynopsis',
                                            'TR.MnAAcquirorPriHiTech',
                                            'TR.MnAAcquirorPriTickerSym',
                                            'TR.MnAAcquirorExchTicker(Concat="|")',
                                            'TR.MnAAcquirorUltParentPermID',
                                            'TR.MnAHiTechGroup(Concat="|")',
                                            'TR.MnaAcquirorDate1DayPriorToAnn',
                                            'TR.MnaAcquirorStockPrice1DayPriorToAnnInAcqCurr(Curn=USD),
                                            'TR.MnAAcquirorStockPriceOnAnn(Curn=USD)',
                                            'TR.MnAAcquirorStockPrice1DayPostAnn(Curn=USD)',
                                            'TR.MnASnP5001DayPreAnn',
                                            'TR.MnASnP500AtAnnouncement',
                                            'TR.MnASnP5001DayPostAnn',
                                            'TR.MnATargetSDCCusip',
                                            'TR.MnAAcquirorSDCCusip',
                                            'TR.MnAAcquirorUltParentSDCCusip',
                                            'TR.MnAValueOfCash(Curn=USD, Scale=6)',
                                            'TR.MnAValueOfEarnout(Curn=USD, Scale=6)',
                                            'TR.MnADealValue(Curn=USD, Scale=6)',
                                            'TR.MnAAcquirorHiTech(Concat="|")',
                                            'TR.MnAAcquirorUltParentHiTech(Concat="|")',
                                            'TR.MnATargetPriHiTech',
                                            'TR.MnAAcquirorMajorInd',
                                            'TR.MnAAcquirorMajorIndGrp',
                                            'TR.MnAAcquirorPriVEIC',
                                            'TR.MnAMasterDealType',
                                            'TR.MnADealType',
                                            'TR.MnAType(Concat="|")',
                                            'TR.MnAAcquirorVeic(Concat="|")',
                                            'TR.MnAPurposeDescription',
                                            'TR.MnAIsCrossBorder',
                                            'TR.MnAAcquirorIsVentureFirm',
                                            'TR.MnATargetPriVEIC',
                                            'TR.MnATargetVeic(Concat="|")',
                                            'TR.MnAAcquirorBusDesc',
                                            'TR.MnATargetBusDesc',
                                            'TR.MnATargetExchTicker(Concat="|")',
                                            'TR.MnATargetUltParentSDCCusip',
                                            'TR.MnATargetUltParentPriTickerSym',
                                            'TR.MnATargetUltParentPermID',
                                            'TR.MnAAcquirorUltParentPriTickerSym',
                                            'TR.MnAAcquirorUltParentExchTicker(Concat="|")',
                                            'TR.MnATargetUltParentExchTicker(Concat="|")',
                                            'TR.MnATargetMajorInd',
                                            'TR.MnATargetMajorIndGrp',
                                            'TR.MnATargetHiTech(Concat="|")',
                                            'TR.MnAAcquirorUltParentPriHiTech',
                                            'TR.MnAAcquirorHiTechGroup(Concat="|")',
                                            'TR.MnATargetHiTechGroup(Concat="|")',
                                            'TR.MnAWithdrawnDate',
                                            'TR.MnADateUnconditional',
                                            'TR.MnAOrigAnnDate',
                                            'TR.MnAPercentCash',
                                            'TR.MnADealCurrency',
                                            'TR.MnAEarnoutDesc']) 

            print(mnas.shape)

#             print("Uploading data to BQ...")
#             mnas.columns = [col[3:].lower() for col in mnas.columns]

#             # Load the DataFrame into BigQuery
#             load_job = client.load_table_from_dataframe(mnas, table_ref, job_config=job_config)

#             # Wait for the load job to complete
#             load_job.result()

#             print("Data appended successfully.")

            master_df = pd.concat([master_df, mnas], ignore_index=True)
            print(f'Master shape: {master_df.shape}')
            print(master_df['TR.MNASDCDealNumber'])

            success = True 
            
        except Exception as e:
            print(f"An error occurred: {e}. Retrying for the range {start} to {end}...")

Start Date: 20180101, End Date: 20180106
(978, 74)
Master shape: (978, 74)
    TR.MNASDCDealNumber TR.MNASDCDealNumber
0            2696976040          2696976040
1            2951155040          2951155040
2            3081061020          3081061020
3            3104001040          3104001040
4            3107974040          3107974040
..                  ...                 ...
973          4043055020          4043055020
974          4043106020          4043106020
975          4122965020          4122965020
976          4125712020          4125712020
977          4148128020          4148128020

[978 rows x 2 columns]
Start Date: 20180107, End Date: 20180112
(1103, 74)
Master shape: (2081, 74)
     TR.MNASDCDealNumber TR.MNASDCDealNumber
0             2696976040          2696976040
1             2951155040          2951155040
2             3081061020          3081061020
3             3104001040          3104001040
4             3107974040          3107974040
...                  ...  

ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 20180119 to 20180124...
Start Date: 20180119, End Date: 20180124


ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 20180119 to 20180124...
Start Date: 20180119, End Date: 20180124
(858, 74)
Master shape: (3824, 74)
     TR.MNASDCDealNumber TR.MNASDCDealNumber
0             2696976040          2696976040
1             2951155040          2951155040
2             3081061020          3081061020
3             3104001040          3104001040
4             3107974040          3107974040
...                  ...                 ...
3819          3671954040          3671954040
3820          3929770040          3929770040
3821          4124146040          4124146040
3822          4161506020          4161506020
3823          4223907040          4223907040

[3824 rows x 2 columns]
Start Date: 20180125, End Date: 20180130
(876, 74)
Master shape: (4700, 74)
     TR.MNASDCDealNumber TR.MNASDCDealNumber
0             2696976040          2696976040
1             2951155040          2951155040
2             3081061020          3081061020
3             3104001040          3

...

An error occurred: . Retrying for the range 20181209 to 20181214...
Start Date: 20181209, End Date: 20181214
(1079, 74)
Master shape: (54365, 74)
      TR.MNASDCDealNumber TR.MNASDCDealNumber
0              2696976040          2696976040
1              2951155040          2951155040
2              3081061020          3081061020
3              3104001040          3104001040
4              3107974040          3107974040
...                   ...                 ...
54360          4095762040          4095762040
54361          4111876020          4111876020
54362          4156788020          4156788020
54363          4200810020          4200810020
54364          4217747020          4217747020

[54365 rows x 2 columns]
Start Date: 20181215, End Date: 20181220
(1175, 74)
Master shape: (55540, 74)
      TR.MNASDCDealNumber TR.MNASDCDealNumber
0              2696976040          2696976040
1              2951155040          2951155040
2              3081061020          3081061020
3             

ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


An error occurred: . Retrying for the range 20181227 to 20190101...
Start Date: 20181227, End Date: 20190101
(759, 74)
Master shape: (56935, 74)
      TR.MNASDCDealNumber TR.MNASDCDealNumber
0              2696976040          2696976040
1              2951155040          2951155040
2              3081061020          3081061020
3              3104001040          3104001040
4              3107974040          3107974040
...                   ...                 ...
56930          4177297040          4177297040
56931          4242721040          4242721040
56932          4261800040          4261800040
56933          4267565040          4267565040
56934          4270805040          4270805040

[56935 rows x 2 columns]
Start Date: 20190102, End Date: 20190107
(662, 74)
Master shape: (57597, 74)
      TR.MNASDCDealNumber TR.MNASDCDealNumber
0              2696976040          2696976040
1              2951155040          2951155040
2              3081061020          3081061020
3              3

KeyboardInterrupt: 

ERROR!!! An error occurred while requesting URL('https://api.refinitiv.com/data/datagrid/beta1/').
     ReadTimeout('')


In [9]:
# Save/load deals dataframe
# master_df.to_csv('refi_deals_2018.csv')
# master_df = pd.read_csv('refi_deals_2018.csv')

# Note: Watch out for currency. I have some mistake. 
#     master_df[(master_df['TR.MnAAcquirorPriTickerSym'] != "") & (master_df['TR.MnAAcquirorNation'] == "United States")] [['TR.MnAAcquirorPriTickerSym', 'TR.MnAAcquirorUltParentPriTickerSym', 'TR.MnAAnnDate', 'TR.MnAOrigAnnDate', 'TR.MnAAcquirorStockPriceOnAnn', 'TR.MnAAcquirorNation']]


In [36]:
# Create list of acquiror tickers from deals df to pass into next query

tickers = master_df[
    (master_df['TR.MnAAcquirorPriTickerSym'] != "") & 
    (master_df['TR.MnAAcquirorPriTickerSym'].notna()) 
    # (master_df['TR.MnAAcquirorNation'] == "United States")
][['TR.MnAAcquirorPriTickerSym']]

print(tickers)

ticker_list = tickers['TR.MnAAcquirorPriTickerSym'].tolist()
ticker_list

      TR.MnAAcquirorPriTickerSym
1                           BDMN
2                         BBU.UN
10                         COLUM
13                          1202
16                           ARW
...                          ...
58353                        ACK
58356                       PENG
58361                     002675
58374                       6782
58378                        JEF

[16380 rows x 1 columns]


['BDMN',
 'BBU.UN',
 'COLUM',
 '1202',
 'ARW',
 'MARKAZ',
 'SWALA',
 '3323',
 'ASUR',
 'ASUR',
 'ASUR',
 'SECU B',
 'CMC',
 'CWN',
 'AROC',
 'GTLS',
 'ALLE',
 'MHC',
 'TECH',
 '300441',
 '600708',
 '600708',
 'POL',
 'PPG',
 'AGRO',
 'AAPL',
 'RQIH',
 '504',
 '1B1',
 'RECI B',
 'RAY.A',
 'D',
 'MNGA',
 'CUDA',
 'DXPE',
 'WAR',
 'REG',
 'GROM',
 'TOVA.P',
 '500304',
 'WSC',
 'BLZ',
 'FNOV17',
 '7169',
 'P8A',
 '004990',
 '002572',
 '300439',
 '3470',
 '004990',
 'AI',
 'SECG',
 'EKHO',
 '0656',
 'MT',
 'SSFT',
 '2456',
 'FSV',
 'WSP',
 'DRAL',
 'LHS',
 'NDSN',
 'CIW',
 'CWST',
 'SEEHUP',
 'WRE',
 'JWCA.H',
 'RTIX',
 'MCLD',
 'SAM',
 '002165',
 '002651',
 '000155',
 'SRF',
 'U19',
 'VMX',
 'HDG',
 '82',
 '3974',
 'TRMB',
 '4502',
 '601933',
 'REP',
 'BVI',
 'BBU.UN',
 '6457',
 'ACB',
 '300003',
 'ERF',
 'ERF',
 'HT',
 'MVEN',
 'RFN',
 'AVX',
 'FELE',
 'VZ',
 'T',
 '030270',
 'ISLA',
 '300340',
 '002133',
 '300202',
 'BX',
 '002043',
 '851',
 'RSTA',
 'KZOO',
 '002362',
 '603939',
 'ARE',

In [43]:

# Get symbols for a given ticker. Need RIC to extract stock price data from refinitiv
# rdp.convert_symbols(
#     ['APO', 'ARW', 'ASUR'], 
#     from_symbol_type='TICKER') #TICKER


# Get symbols for all tickers. Need to process in chunks to avoid refinitiv's api to time out
chunk_results = []
chunk_size = 500

for start in range(0, len(ticker_list), chunk_size):
    end = min(start + chunk_size, len(ticker_list))
    print(f'{start}:{end}')
    ticker_chunk = ticker_list[start:end]
    chunk_result = rdp.convert_symbols(
        ticker_chunk, 
        from_symbol_type='TICKER'
    )
    chunk_results.append(chunk_result)
    print(chunk_results)
    
# Concatenate all chunk results into a final DataFrame
symbols = pd.concat(chunk_results, ignore_index=True)
symbols

0:500
[       TickerSymbol      CUSIP IssuerOAPermID  \
BDMN           BDMN       <NA>     4295874431   
BBU.UN       BBU.UN       <NA>     5046722004   
COLUM         COLUM       <NA>     4295866048   
1202           1202       <NA>     4295864118   
ARW             ARW  042735100     4295903472   
...             ...        ...            ...   
LXI             LXI  526107107     4295912186   
MGP             MGP       <NA>           <NA>   
USAC           USAC  90290N109     4296123182   
RDI             RDI  755408101     4295915527   
CCL.A         CCL.A  124900200     4295860622   

                                            DocumentTitle    SEDOL       RIC  \
BDMN    Bank Danamon Indonesia Tbk PT, Ordinary Share,...  6580034   BDMN.JK   
BBU.UN  Brookfield Business Partners LP, Unit, The Tor...  BYNDNQ0  BBU_u.TO   
COLUM   Columbus A/S, Ordinary Share, OMX Nordic Excha...  5960916  COLUM.CO   
1202    Chengdu SIWI Science and Technology Co Ltd, Or...  6203546   1202.HK   
ARW 

,TickerSymbol,CUSIP,IssuerOAPermID,DocumentTitle,SEDOL,RIC,IssueISIN,FundClassLipperID
0,BDMN,<NA>,4295874431,"Bank Danamon Indonesia Tbk PT, Ordinary Share,...",6580034,BDMN.JK,ID1000094204,<NA>
1,BBU.UN,<NA>,5046722004,"Brookfield Business Partners LP, Unit, The Tor...",BYNDNQ0,BBU_u.TO,BMG162341090,68578481
2,COLUM,<NA>,4295866048,"Columbus A/S, Ordinary Share, OMX Nordic Excha...",5960916,COLUM.CO,DK0010268366,<NA>
3,1202,<NA>,4295864118,"Chengdu SIWI Science and Technology Co Ltd, Or...",6203546,1202.HK,CNE1000002C2,<NA>
4,ARW,042735100,4295903472,"Arrow Electronics Inc, Ordinary Share, NYSE Co...",<NA>,ARW,US0427351004,<NA>
...,...,...,...,...,...,...,...,...
13630,SDLP,<NA>,5037847419,"Seadrill Partners Units, Unit, Delisted, Chica...",<NA>,SDLP.W^E14,MHY7545W1259,<NA>
13631,ACK,<NA>,4295880278,"Fuji Oil Co Ltd, Ordinary Share, Frankfurt Sto...",7552184,5017.F,JP3160300004,<NA>
13632,PENG,<NA>,<NA>,"AI-PENG, Open-End Fund, Delisted, Oslo Stock E...",<NA>,AI-PENG.OL^L07,<NA>,<NA>
13633,6782,<NA>,5035300917,"Visco Vision Inc, Ordinary Share, Taiwan Stock...",BM9WRL0,6782.TW,TW0006782006,<NA>


In [203]:
# Bring symbols back to main dataframe, merge on ticker.

# Perform a left merge on the 'TickerSymbol' column
merged_df = pd.merge(master_df, symbols, how='left', left_on='TR.MnAAcquirorPriTickerSym', right_on='TickerSymbol')
merged_df = merged_df.drop_duplicates(subset=['TR.MNASDCDealNumber'], keep='first') # Drop duplicate deals, I didn't find the source of duplicates

# Display the resulting merged DataFrame
merged_df

,Unnamed: 0,instrument,TR.MNASDCDealNumber,TR.MnAAnnDate,TR.MnARankValueIncNetDebt,TR.MnATarget,TR.MnATargetPermId,TR.MnATargetPriTickerSym,TR.MnATargetNation,TR.MnATargetMacroInd,...,TR.MnADealCurrency,TR.MnAEarnoutDesc,TickerSymbol,CUSIP,IssuerOAPermID,DocumentTitle,SEDOL,RIC,IssueISIN,FundClassLipperID
0,0,154081636364,2696976040,2018-01-06,602.623335,Belgrade Nikola Tesla Airport Jsc-Concession,NaN,NaN,Serbia,Industrials,...,Serbian Dinar,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1,154083046842,2951155040,2018-01-02,110.927885,Bank Nusantara Parahyangan Tbk PT,4.295874e+09,BBNP,Indonesia,Financials,...,Indonesian Rupiah,NaN,BDMN,<NA>,4295874431,"Bank Danamon Indonesia Tbk PT, Ordinary Share,...",6580034,BDMN.JK,ID1000094204,<NA>
2,2,154083214825,3081061020,2018-01-04,4600.000000,Westinghouse Electric Co LLC,4.297335e+09,NaN,United States,Energy and Power,...,US Dollar,NaN,BBU.UN,<NA>,5046722004,"Brookfield Business Partners LP, Unit, The Tor...",BYNDNQ0,BBU_u.TO,BMG162341090,68578481
6,3,154083245535,3104001040,2018-01-02,360.923874,AVIC International Vanke Co Ltd,NaN,NaN,China (Mainland),Real Estate,...,Chinese Yuan,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,4,154083250434,3107974040,2018-01-02,NaN,L'Appart SAS,5.056390e+09,NaN,France,Retail,...,Euro,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84577,58379,154087656025,3955160040,2019-01-11,NaN,Jokkmokks Korv AB,5.045247e+09,NaN,Sweden,Consumer Staples,...,US Dollar,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
84578,58380,154088013950,4087420020,2019-01-09,0.750000,TaxBit Inc,5.067500e+09,NaN,United States,High Technology,...,US Dollar,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
84579,58381,154088083578,4099143020,2019-01-08,NaN,Clique Media Inc,5.047746e+09,NaN,United States,High Technology,...,US Dollar,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
84580,58382,154088155027,4129515020,2019-01-09,NaN,Aero Turbine Inc,5.045899e+09,NaN,United States,Industrials,...,US Dollar,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [118]:
# ## This chunk is not used. This is an example code to get daily price data via get_historical... Shares outstanding not available. also, no currency conversion function. I'm using a different function.
# price_hist_example = rdp.get_historical_price_summaries(
#     universe = 'AAPL.O', 
#     interval = rdp.Intervals.DAILY,
#     start = "2024-08-16", 
#     end = "2024-08-16",
#     fields = ['TRDPRC_1', 'ACVOL_UNS',  'OPEN_PRC', 'TRNOVR_UNS',  'NUM_MOVES', 'BLKVOLUM']
# )

# price_hist_example



# ## Example function I use to retrieve stock data for a given date. This allows me to retrieve to shares outstanding, currency == USD
# ## if want native curreny pass: CF_LAST, CF_CURR
# ## totalReturn1D is for the previous day? https://community.developers.refinitiv.com/questions/103917/total-return-for-the-same-day.html

# rdp.get_data(
#     'AAPL.O',
#     fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
#               'TR.VOLUME', 'TR.SharesOutstanding', 'TR.CompanyMarketCap', 
#               'TR.TotalReturn1D'],
#     parameters = {'SDate':'2024-08-10','EDate':'2024-08-16','Curn':'USD'}
# )


# ## Example to get index prices
# dab = rdp.get_data(
#     ['.DJI', '.FTSE', '.SPX'],
#     fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
#               'TR.VOLUME'],
#     parameters = {'SDate':'2024-08-10','EDate':'2024-08-16','Curn':'USD'}
# )
# dab


,TRDPRC_1,ACVOL_UNS,OPEN_PRC,TRNOVR_UNS,BLKVOLUM,NUM_MOVES
2024-08-16,226.05,44340240,223.92,9995644943,17560109,562638


In [ ]:
## Iterate over merged_df to get stock price data for days around acquisition

# Suppress warnings -- reinstated at the end of the loop
warnings.filterwarnings("ignore")

# Ensure 'TR.MnAAnnDate' is in datetime format
merged_df['TR.MnAAnnDate'] = pd.to_datetime(merged_df['TR.MnAAnnDate'])
merged_df['RIC'] = merged_df['RIC'].str.strip()  # Remove leading/trailing spaces

# Initialize an empty DataFrame to store the results
historical_data = pd.DataFrame()
# Initialize an empty DataFrame to store error information
error_log = pd.DataFrame(columns=['Index', 'RIC', 'Error'])

# Counter to track the number of lines processed
counter = 0
# Loop through the merged_df to extract the required variables. Subset on non null RIC values
for index, row in merged_df[merged_df['RIC'].notna() & (merged_df['RIC'] != '')].iterrows():

    RIC = row['RIC']
    MnAAnnDate = row['TR.MnAAnnDate']
    MNASDCDealNumber = row['TR.MNASDCDealNumber']


    # Calculate start and end dates (MnAAnnDate - 10 days and MnAAnnDate + 10 days)
    start_date = (MnAAnnDate - timedelta(days=10)).strftime('%Y-%m-%d')
    end_date = (MnAAnnDate + timedelta(days=10)).strftime('%Y-%m-%d')

    # Fetch historical price summaries
    success = False
    while not success:
        try:

            ric_returns = rdp.get_data(RIC,
                            fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
                                'TR.VOLUME', 'TR.SharesOutstanding', 'TR.CompanyMarketCap', 
                                'TR.TotalReturn1D'],
                            parameters = {'SDate':start_date,
                                          'EDate':end_date,
                                          'Curn':'USD'}
                            )

            # Add additional columns to the result (start, end, MnAAnnDate, RIC)
            ric_returns['Start'] = start_date
            ric_returns['End'] = end_date
            ric_returns['MnAAnnDate'] = MnAAnnDate
            ric_returns['RIC'] = RIC
            ric_returns['MNASDCDealNumber'] = MNASDCDealNumber
            # ric_returns['counter'] = counter

            print(ric_returns)

            # Append the result to the historical_data DataFrame
            historical_data = pd.concat([historical_data, ric_returns], ignore_index=True)

            counter += 1

        # Print progress every 100 lines
            if counter % 100 == 0:
                print(f"Processed {counter} lines")
                
            success = True

        except Exception as e:
            print(f"An error occurred for index: {index}, RIC {RIC}: {e}")
            # Create a DataFrame for the error entry
            error_entry = pd.DataFrame({'Index': [index], 'RIC': [RIC], 'Error': [str(e)]})
            # Concatenate the error entry to the error_log DataFrame
            error_log = pd.concat([error_log, error_entry], ignore_index=True)

        
# Restore warnings
warnings.filterwarnings("default")


# Display the final historical_data DataFrame
print(historical_data)

# Display the error log DataFrame
print(error_log)

In [206]:
historical_data.to_csv('historical_data.csv')